In [5]:
from autoencoders.autoencoder import UserAutoEncoder, ItemAutoEncoder
from utils.data_utils import preprocess_data
from utils.evaluation import calculate_recall_at_k

import wandb

In [6]:
dataset = 'ml_100k'
p_value = 0.1
R_bar_train, R_bar_val, _, _, _, _ = preprocess_data(dataset=dataset ,p=p_value)

In [7]:
import optuna
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

class WandbLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            wandb.log({'loss': logs['loss'], 'val_loss': logs['val_loss']})

# def wandb_callback(study, trial):
#     # Log trial metrics and parameters to wandb
#     wandb.log({
#         'trial_number': trial.number,
#         'trial_value': trial.value,
#         'parameters': trial.params
#     })

def objective(trial, dataset, p_value, model_type, R_bar_train, R_bar_val):
    # Shared hyperparameters
    latent_dim = trial.suggest_categorical('latent_dim', [2, 4, 8, 16, 32, 64, 128, 256])
    num_layers = trial.suggest_int('num_layers', 1, 5)
    regularizer_factor = trial.suggest_categorical('regularizer_factor', [0, 1e-2])
    dropout_rate = trial.suggest_categorical('dropout_rate', [0.0, 0.1, 0.2, 0.3])
    use_batch_norm = trial.suggest_categorical('use_batch_norm', [True, False])

    # Create a unique run name using the trial number and hyperparameters
    run_name = f"dataset_{dataset}, trial_{trial.number}, latent_{latent_dim}, layers_{num_layers}, regularizer_factor_{regularizer_factor}, dropout_rate_{dropout_rate}, use_batch_norm_{use_batch_norm}"
    
    # Initialize a new wandb run for each trial
    wandb.init(project="autoencoder_optimization", name=run_name, reinit=True, 
               config={
                   'dataset': dataset,
                   'p_value': p_value,
                   'latent_dim': latent_dim,
                   'num_layers': num_layers,
                   'regularizer_factor': regularizer_factor,
                   'dropout_rate': dropout_rate,
                   'use_batch_norm': use_batch_norm
               })

    # Initialize the model based on the chosen type
    if model_type == "user":
        autoencoder = UserAutoEncoder(R_bar_train.shape[1], latent_dim, num_layers, regularizer_factor, dropout_rate, use_batch_norm)
    elif model_type == "item":
        autoencoder = ItemAutoEncoder(R_bar_train.shape[0], latent_dim, num_layers, regularizer_factor, dropout_rate, use_batch_norm)
    else:
        raise ValueError("Invalid model type specified")

    # Set up EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Compile and train the model with the WandbLogger callback
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    autoencoder.fit(R_bar_train, R_bar_train, 
                    epochs=1000, 
                    batch_size=256,
                    validation_data=(R_bar_val, R_bar_val), 
                    verbose=2, 
                    callbacks=[WandbLogger(), early_stopping])

    # Generate recommendations and compute recall_at_k
    # Note: Modify this to suit how you generate predictions and evaluate recall for your model
    pred_val = autoencoder.predict(R_bar_val)
    
    k_values = [5, 10, 50, 100]
    avg_recall_at_k, avg_random_recall_at_k = calculate_recall_at_k(R_bar_train, R_bar_val, pred_val, k_values)

    # Log each recall_at_k and random_recall_at_k value as separate columns
    for k in k_values:
        wandb.log({f'recall_at_{k}': avg_recall_at_k[k], f'random_recall_at_{k}': avg_random_recall_at_k[k]})

    wandb.finish()
    
    return -avg_recall_at_k[10]  # Optimize for a specific recall value



In [8]:
# Initialize wandb
# wandb.init(project="autoencoder_optimization")

# Define the hyperparameter grid
search_space = {
    'latent_dim': [2, 4, 8, 16, 32, 64, 128, 256],
    'num_layers': list(range(1, 6)),
    'regularizer_factor': [0, 1e-2],
    'dropout_rate': [0.0, 0.1, 0.2, 0.3],
    'use_batch_norm': [True, False]
}

# Create the study with GridSampler
model_type_to_tune = 'user'

study = optuna.create_study(sampler=optuna.samplers.GridSampler(search_space))
study.optimize(
    lambda trial: objective(trial, dataset, p_value, model_type_to_tune, R_bar_train, R_bar_val),
    n_trials=None,  # Allow Optuna to run through all combinations in the grid
)
# Close wandb run
# wandb.finish()

[I 2023-12-16 19:54:55,513] A new study created in memory with name: no-name-fe3c2021-fea1-4408-bd08-03f3591b1627


Epoch 1/1000
4/4 - 2s - loss: 6.6627 - val_loss: 5.5493 - 2s/epoch - 575ms/step
Epoch 2/1000
4/4 - 1s - loss: 4.9958 - val_loss: 4.2379 - 975ms/epoch - 244ms/step
Epoch 3/1000
4/4 - 1s - loss: 3.7705 - val_loss: 3.2756 - 1s/epoch - 251ms/step
Epoch 4/1000
4/4 - 1s - loss: 3.0114 - val_loss: 2.6968 - 1s/epoch - 252ms/step
Epoch 5/1000
4/4 - 1s - loss: 2.4996 - val_loss: 2.2854 - 987ms/epoch - 247ms/step
Epoch 6/1000
4/4 - 2s - loss: 2.1495 - val_loss: 1.9290 - 2s/epoch - 382ms/step
Epoch 7/1000
4/4 - 1s - loss: 1.8838 - val_loss: 1.6627 - 1s/epoch - 298ms/step
Epoch 8/1000
4/4 - 1s - loss: 1.6674 - val_loss: 1.4595 - 1s/epoch - 308ms/step
Epoch 9/1000
4/4 - 1s - loss: 1.4799 - val_loss: 1.2850 - 1s/epoch - 372ms/step
Epoch 10/1000
4/4 - 1s - loss: 1.3193 - val_loss: 1.1284 - 1s/epoch - 349ms/step
Epoch 11/1000
4/4 - 1s - loss: 1.1812 - val_loss: 0.9991 - 1s/epoch - 286ms/step
Epoch 12/1000
4/4 - 1s - loss: 1.0664 - val_loss: 0.8975 - 1s/epoch - 304ms/step
Epoch 13/1000
4/4 - 1s - loss: 

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
random_recall_at_10,▁
random_recall_at_100,▁
random_recall_at_5,▁
random_recall_at_50,▁
recall_at_10,▁
recall_at_100,▁
recall_at_5,▁
recall_at_50,▁
val_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,0.15387


[I 2023-12-16 20:02:03,853] Trial 0 finished with value: -0.056163947160317085 and parameters: {'latent_dim': 8, 'num_layers': 2, 'regularizer_factor': 0.01, 'dropout_rate': 0.1, 'use_batch_norm': False}. Best is trial 0 with value: -0.056163947160317085.


Error: You must call wandb.init() before wandb.log()